# 3. Testing Modern LPMs

Consider the following factor models:
* CAPM: MKT
* Fama-French 3F: MKT, SMB, HML
* Fama-French 5F: MKT, SMB, HML, RMW, CMA
* AQR: MKT, HML, RMW, UMD

Our labeling of the last model as the **AQR** is just for concreteness. The firm is well-known for these factors and an unused case study discusses that further.

For instance, for the AQR model is...

$$
\mathbb{E}[\tilde{r}^i] 
= \beta^{i,\mathrm{MKT}} \, \mathbb{E}[\tilde{f}^{\mathrm{MKT}}] 
+ \beta^{i,\mathrm{HML}} \, \mathbb{E}[\tilde{f}^{\mathrm{HML}}] 
+ \beta^{i,\mathrm{RMW}} \, \mathbb{E}[\tilde{f}^{\mathrm{RMW}}] 
+ \beta^{i,\mathrm{UMD}} \, \mathbb{E}[\tilde{f}^{\mathrm{UMD}}]
$$

We will test these models with the time-series regressions. Namely, for each asset i, estimate the following regression to test the AQR model:

$$
\tilde{r}^i_t 
= \alpha^i 
+ \beta^{i,\mathrm{MKT}} \tilde{f}^{\mathrm{MKT}}_t 
+ \beta^{i,\mathrm{HML}} \tilde{f}^{\mathrm{HML}}_t 
+ \beta^{i,\mathrm{RMW}} \tilde{f}^{\mathrm{RMW}}_t 
+ \beta^{i,\mathrm{UMD}} \tilde{f}^{\mathrm{UMD}}_t 
+ \varepsilon_t
$$

### Data

* Monthly excess return data on `n=49` equity portfolios sorted by their industry. Denote these as $\tilde{r}^i$ , for $n = 1, . . . .$

* You do NOT need the risk-free rate data. It is provided only for completeness. The other two tabs are already in terms of excess returns.

In [31]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pathlib import Path

file_path = Path("factor_pricing_data_monthly.xlsx")
xls = pd.ExcelFile(file_path)

for s in xls.sheet_names:
    try:
        df = pd.read_excel(xls, sheet_name=s)
    except Exception as e:
        print("Error", s, ":", e)

In [32]:
factors_sheet = "factors (excess returns)"        
returns_sheet = "portfolios (excess returns)"     

factors = pd.read_excel(xls, sheet_name=factors_sheet, parse_dates=True)
returns = pd.read_excel(xls, sheet_name=returns_sheet, parse_dates=True)

for df in (factors, returns):
    if not isinstance(df.iloc[:,0].dtype, (np.dtype,)):
        pass

date_cols = ['date', 'Date', 'DATE', 'Month', 'PERIOD', 'Period']
def set_date_index(df):
    for col in date_cols:
        if col in df.columns:
            df2 = df.copy()
            df2[col] = pd.to_datetime(df2[col])
            df2 = df2.set_index(col)
            return df2
    try:
        df2 = df.copy()
        df2.iloc[:,0] = pd.to_datetime(df2.iloc[:,0])
        df2 = df2.set_index(df2.columns[0])
        return df2
    except Exception as e:
        raise ValueError(str(e))

factors = set_date_index(factors)
returns = set_date_index(returns)

print("factors shape:", factors.shape)
print("returns shape:", returns.shape)
display(factors.head())
display(returns.iloc[:, :10].head())  

factors shape: (548, 6)
returns shape: (548, 49)


,MKT,SMB,HML,RMW,CMA,UMD
Date,,,,,,
1980-01-31,0.0550,0.0188,0.0185,-0.0184,0.0189,0.0745
1980-02-29,-0.0123,-0.0162,0.0059,-0.0095,0.0292,0.0789
1980-03-31,-0.1289,-0.0697,-0.0096,0.0182,-0.0105,-0.0958
1980-04-30,0.0396,0.0105,0.0103,-0.0218,0.0034,-0.0048
1980-05-31,0.0526,0.0200,0.0038,0.0043,-0.0063,-0.0118


,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths
Date,,,,,,,,,,
1980-01-31,-0.0073,0.0285,0.0084,0.1009,-0.0143,0.0995,0.0348,0.0323,0.0048,0.0059
1980-02-29,0.0125,-0.0609,-0.0967,-0.0323,-0.0575,-0.0316,-0.0492,-0.0803,-0.0556,-0.0169
1980-03-31,-0.2220,-0.1119,-0.0158,-0.1535,-0.0188,-0.1272,-0.0827,-0.1238,-0.0567,-0.0670
1980-04-30,0.0449,0.0767,0.0232,0.0289,0.0830,-0.0529,0.0785,0.0154,0.0305,0.0115
1980-05-31,0.0635,0.0797,0.0458,0.0866,0.0822,0.0510,0.0325,0.0888,0.0560,0.0098


In [33]:
data = returns.join(factors, how='inner')  
print("Merged shape:", data.shape)

default_factor_names = ['MKT', 'SMB', 'HML', 'RMW', 'CMA', 'UMD'] 
factor_cols = [c for c in factors.columns if any(k in c.upper() for k in [f.upper() for f in default_factor_names])]
if len(factor_cols)==0:
    factor_cols = list(factors.columns)

asset_cols = [c for c in returns.columns if c not in factor_cols]
print("Detected factor columns:", factor_cols)
print("Detected asset columns (preview):", asset_cols[:10])

Merged shape: (548, 55)
Detected factor columns: ['MKT', 'SMB', 'HML', 'RMW', 'CMA', 'UMD']
Detected asset columns (preview): ['Agric', 'Food ', 'Soda ', 'Beer ', 'Smoke', 'Toys ', 'Fun  ', 'Books', 'Hshld', 'Clths']


In [34]:
models = {
    'CAPM': ['MKT'],
    'FF3': ['MKT','SMB','HML'],
    'FF5': ['MKT','SMB','HML','RMW','CMA'],
    'AQR4': ['MKT','HML','RMW','UMD']
}

for m, facs in models.items():
    missing = [f for f in facs if not any(f.upper() in c.upper() for c in factor_cols)]
    if missing:
        print(f"The following factors for model {m} were not explicitly found in the automatically detected factor_cols: {missing}")
        print("detected factor_cols:", factor_cols)

### 1. 

Test the AQR 4-Factor Model using the time-series test. (We are not doing the cross-sectional regression tests.)

For each regression, report the estimated α and r-squared.

In [41]:
def ts_regression(y, X, add_const=True):
    if add_const:
        X = sm.add_constant(X)
    model = sm.OLS(y, X, missing='drop')
    res = model.fit()
    return res

def run_time_series_for_model(model_name, factor_list, data_df, asset_columns):
    results = {}
    r2_list = []
    alpha_list = []
    alpha_t_list = []
    betas = pd.DataFrame(index=asset_columns, columns=['const'] + factor_list, dtype=float)
    for asset in asset_columns:
        y = data_df[asset].astype(float)
        X = data_df[[c for c in factor_cols if any(f.upper() in c.upper() for f in factor_list)]].astype(float)
        cols_for_model = []
        for f in factor_list:
            matches = [c for c in X.columns if f.upper() in c.upper()]
            if not matches:
                raise KeyError(f"Column {f} not found among available factors: {X.columns.tolist()}")
            cols_for_model.append(matches[0])  
        X_model = X[cols_for_model]
        res = ts_regression(y, X_model, add_const=True)
        r2_list.append(res.rsquared)
        alpha = res.params.get('const', np.nan)
        alpha_list.append(alpha)
        for f, cname in zip(['const']+factor_list, ['const']+cols_for_model):
            if f == 'const':
                betas.at[asset, 'const'] = res.params.get('const', np.nan)
            else:
                betas.at[asset, f] = res.params.get(cols_for_model[factor_list.index(f)], np.nan) if f in factor_list else np.nan
        results[asset] = res
    summary = pd.DataFrame({
        'alpha': alpha_list,
        'r_squared': r2_list
    }, index=asset_columns)
    return results, summary, betas

time_series_summaries = {}
time_series_results = {}
time_series_betas = {}
for mname, facs in models.items():
    res_dict, summary_df, betas_df = run_time_series_for_model(mname, facs, data, asset_cols)
    time_series_results[mname] = res_dict
    time_series_summaries[mname] = summary_df
    time_series_betas[mname] = betas_df

aqr_summary = time_series_summaries['AQR4']
display(aqr_summary)

,alpha,r_squared
Agric,0.000971,0.342074
Food,0.000125,0.455064
Soda,0.001282,0.302544
Beer,0.000821,0.414773
Smoke,0.003426,0.265363
Toys,-0.002809,0.510213
Fun,0.003255,0.607213
Books,-0.003059,0.688933
Hshld,-0.001062,0.554712
Clths,-0.001889,0.618968


### 2. 

Calculate the mean-absolute-error of the estimated alphas.

$$\text{MAE} = \frac{1}{n}\sum_{i=1}^n|\tilde{\alpha}^i|$$

* If the pricing model worked, should these alpha estimates be large or small? Why?

* Based on your MAE stat, does this seem to support the pricing model or not?

In [61]:
print("The mean-absolute-error of the estimated alphas:", aqr_summary['alpha'].abs().mean())

The mean-absolute-error of the estimated alphas: 0.0020509040868031477


* If the pricing model worked, these alpha estimates should be small. Because alpha represents the average excess return that cannot be explained by the model; a pricing model capable of explaining the average return of an asset would attribute these “abnormal” returns to factors, so alpha should approach zero.
* The average absolute alpha of approximately 0.21% per month is relatively small, indicating that the model generally performs well over the time series.

### 2. 

Test the CAPM, FF 3-Factor Model and the the FF 5-Factor Model.
   * Report the MAE statistic for each of these models and compare it with the AQR Model MAE.
   * Which model fits best?

In [62]:
mae_by_model = {}
for mname, summ in time_series_summaries.items():
    mae = summ['alpha'].abs().mean()
    mae_by_model[mname] = mae

mae_df = pd.DataFrame.from_dict(mae_by_model, orient='index', columns=['MAE_alpha']).sort_values('MAE_alpha')
display(mae_df)
print("Best-fitting model (by MAE_alpha):", mae_df.index[0])

,MAE_alpha
CAPM,0.001748
FF3,0.002030
AQR4,0.002051
FF5,0.002614


Best-fitting model (by MAE_alpha): CAPM


### 3. 

Does any particular factor seem especially important or unimportant for pricing? Do you think Fama and French should use the Momentum Factor?

Momentum helps reduce unexplained alphas, so it should be included as an additional pricing factor.

### 4. 

This does not matter for pricing, but report the average (across $n$ estimations) of the time-series regression r-squared statistics.
   * Do this for each of the three models you tested.
   * Do these models lead to high time-series r-squared stats? That is, would these factors be good in a Linear Factor Decomposition of the assets?

In [63]:
avg_r2 = {m: time_series_summaries[m]['r_squared'].mean() for m in time_series_summaries}
avg_r2_df = pd.DataFrame.from_dict(avg_r2, orient='index', columns=['avg_r_squared']).sort_values('avg_r_squared', ascending=False)
display(avg_r2_df)

,avg_r_squared
FF5,0.591768
AQR4,0.571935
FF3,0.567874
CAPM,0.522622


Yes. The models produce moderately high time-series R² values (around 0.52–0.59), meaning they explain a large portion of the variation in portfolio returns.

### 5. 

We tested three models using the time-series tests (focusing on the time-series alphas.) Re-test these models, but this time use the cross-sectional test.

* Report the time-series premia of the factors (just their sample averages,) and compare to the cross-sectionally estimated premia of the factors. Do they differ substantially?
* Report the MAE of the cross-sectional regression residuals for each of the four models. How do they compare to the MAE of the time-series alphas?

#### Footnote:

Recall that we found in `Homework 4` that the market premium went from being strongly positive to strongly negative when estimated in the cross-section.

In [ ]:
cs_results = {}
for mname, facs in models.items():
    betas_df = time_series_betas[mname]
    beta_cols = [f for f in facs]
    beta_matrix = pd.DataFrame(index=betas_df.index)
    for f in facs:
        matches = [c for c in betas_df.columns if f.upper() in str(c).upper()]
        if matches:
            beta_matrix[f] = betas_df[matches[0]].astype(float)
        elif f in betas_df.columns:
            beta_matrix[f] = betas_df[f].astype(float)
        else:
            beta_matrix[f] = np.nan

    mean_r = data[asset_cols].mean(axis=0)
    valid = beta_matrix.dropna().index.intersection(mean_r.dropna().index)
    X = beta_matrix.loc[valid].astype(float)
    y = mean_r.loc[valid].astype(float)
    X_cs = sm.add_constant(X)
    cs_mod = sm.OLS(y, X_cs).fit()
    fitted = cs_mod.fittedvalues
    residuals = y - fitted
    mae_resid = residuals.abs().mean()

    time_series_premia = {}
    for f in facs:
        matches = [c for c in data.columns if f.upper() in c.upper()]
        if matches:
            time_series_premia[f] = data[matches[0]].mean()
        elif f in data.columns:
            time_series_premia[f] = data[f].mean()
        else:
            time_series_premia[f] = np.nan

    comparison_df = pd.DataFrame({
        'time_series_premia': pd.Series(time_series_premia),
        'cross_section_premia': cs_mod.params.drop('const', errors='ignore')
    })
    
    cs_results[mname] = {
        'comparison_df': comparison_df,
        'cs_model': cs_mod,
        'lambdas': cs_mod.params,
        'mae_resid': mae_resid,
        'y': y,
        'fitted': fitted,
        'residuals': residuals
    }

    print(f"===== {mname} Model =====")
    display(comparison_df)
    print(f"MAE of cross-sectional residuals: {mae_resid:.6f}\n")

===== CAPM Model =====


,time_series_premia,cross_section_premia
MKT,0.007296,0.00066


MAE of cross-sectional residuals: 0.001267

===== FF3 Model =====


,time_series_premia,cross_section_premia
MKT,0.007296,0.003237
SMB,0.000510,-0.003302
HML,0.002170,-0.001753


MAE of cross-sectional residuals: 0.001000

===== FF5 Model =====


,time_series_premia,cross_section_premia
MKT,0.007296,0.003360
SMB,0.000510,-0.003452
HML,0.002170,-0.002162
RMW,0.003671,0.001559
CMA,0.002357,-0.001840


MAE of cross-sectional residuals: 0.000998

===== AQR4 Model =====


,time_series_premia,cross_section_premia
MKT,0.007296,0.001433
HML,0.002170,-0.002688
RMW,0.003671,0.001462
UMD,0.005026,0.000024


MAE of cross-sectional residuals: 0.001134



In [66]:
cs_mae = {m: cs_results[m]['mae_resid'] for m in cs_results}
cs_mae_df = pd.DataFrame.from_dict(cs_mae, orient='index', columns=['MAE_cs_residual'])
compare_df = mae_df.join(cs_mae_df)
display(compare_df)

,MAE_alpha,MAE_cs_residual
CAPM,0.001748,0.001267
FF3,0.002030,0.001000
AQR4,0.002051,0.001134
FF5,0.002614,0.000998
